In [1]:
import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

system_context = """
You are a medical assistant trained to extract **tumor size measurements** from Mohs patient notes.

Your task is to identify and return **each distinct tumor size** mentioned in the note, based on available clinical language.

Tumor size should be output as a list of strings, each representing a **2-dimensional measurement** in one of the following formats, ranked from best to fallback:

1. "Acm x Bcm" or "Amm x Bmm" (e.g., "1.2cm x 0.8cm") ← Best format  
2. "A x Bcm" or "A x Bmm" (e.g., "1 x 0.7cm") ← Acceptable fallback  
3. "A x B" (e.g., "0.8 x 0.5") ← Acceptable if no units are available  
4. "Acm" or "Amm" ← Use only if single measurement is given  
5. "A" ← Use only if no units or dimensions are provided

---

**Instructions:**

- Capture **each measurement** mentioned (there is likely more than one in the note).
- If a measurement uses **units** (cm or mm), preserve them.
- If no unit is provided, assume the note is ambiguous and fall back to format #3, #4, or #5 based on available data.
- If the size is given as a range (e.g., "0.7–0.9cm"), extract it as "0.7cm x 0.9cm".
- Ignore references to excision size, defect size, or repair dimensions unless it clearly refers to tumor size.
- Only include **tumor sizes**, not scar, flap, or graft sizes.

---

**Output format:**
A Python-style list of strings. Example:
["1.2cm x 0.8cm", "0.9 x 0.6cm", "0.5", "7mm x 1.4cm"]

Do not add comments, explanations, or anything outside the list.
"""

def call_gpt(query):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    
    # Payload for the request
    payload = {
    "messages" : [
            {"role": "system", "content": system_context},
            {"role": "user", "content": f"{query}"},
        ],
      "temperature": 0.7,
      "top_p": 0.95,
      "max_tokens": 800
    }
    
    
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        print("Response Content:", response.text)
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    #print(response.json())
    #return response.json().choices[0].message.content
    return response.json()["choices"][0]["message"]["content"]

In [14]:
# Actual sizes for first 10 patients:

# 36 - (0.8 x 0.8 cm, 0.4 x 0.5 cm), 240 - (0.6 x 0.4 cm), 32 - (3 x 3 mm) <- sometimes ignored due to units given in mm and not cm
# 111 - (6.5 x 5 cm), 197 - (0.6 x 0.8 cm, 1.2 x 1.6 cm, 1 x 1 cm, 2.5 x 3.5 cm), 60 - (0.4 x 0.3 cm)
# 65 - (0.3 x 0.3) <- no units given in notes, sometimes ignored, 215 - (0.5 cm, 8 mm, 8 x 5 mm), 301 - (1 x 0.7 cm, 1.5 x 1, 1 x 1.5 cm)
# 67 - (1.5 cm x 1 cm, 8 mm x 5 mm)

# Notes:
# Run 1st code block to be able to run anything else
# sampleSize = 5 means the first 5 patients are being read through

# Concerns:
# Some Physical Exam notes don't mention tumor sizes, set those to output as 0

# A result from the Size at Greatest Dimension column was wrong, could be true for other results in the column,
# this means the accuracy is higher than what it calculated, for example:
# Result[24]'s largest dimension is 0.4 cm, not 0.3 cm, column in sheet is incorrect

In [2]:
# read in full PE notes
import pandas as pd
Cat = pd.read_excel(r'tumor_size_all.xlsx',sheet_name='Sheet1',header=0) # Only Size at Greatest Dimension column
#Cat = pd.read_excel(r'De-Identified Mohs Full Data.xlsx',sheet_name='Sheet1',header=0) # All columns
Cat.head()

#input_columns = Cat.loc[46, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
#pd.set_option('display.max_colwidth', None)
#print(input_columns)

# len(Cat) = 294

,Randomized Study Number,Size at Greatest Dimension (S) ***Be sure this is from the note and not the Mohs episode!,Physical Exam (PE),Assessment & Plan (A&P),Addendum (A)
0,36,0.8,"Physical exam:\nGeneral: Well developed, well ...",1. Chronic actinic damage in the setting of hi...,Addendum 7/11/2023\n \nThe pathology results w...
1,240,0.6,Physical exam:\nGeneral: Well-dressed; well-no...,Procedure Orders\nBiopsy [486623795] ordered b...,Addendum 7/11/2023\nThe pathology results were...
2,32,0.3,Physical exam:\nGeneral: Well-dressed; well-no...,Assessment & Plan: \n1. Brown papule on the le...,Addendum 7/13/2023\n \nThe pathology results w...
3,111,6.5,Physical exam:\nVital signs: There were no vit...,Assessment & Plan: \nBiopsy-proven basal cell ...,NaN
4,197,3.5,"Physical exam:\nGeneral: Well developed, well ...",Assessment & Plan: \n1. Chronic actinic sun d...,Addendum 7/5/2023\n \nThe pathology results we...


In [3]:
# Set options here
sampleSize = len(Cat) # Change number to increase or decrease number of patients read from file
#sampleSize = 10
multiple_columns = True # Set to True to use all input text columns, False to use only PE column

# Array of indexes containing patients with notes that have problems
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = True # Set to True to skip bad patient notes, False to keep them in
# 11, 33, 45, 66, 132 have answer in A or A&P

In [4]:
# Takes the text in the PE, A&P, and A columns and asks ChatGPT to return an array of all of the tumor sizes
# There may be multiple numbers due to it being a dimension so each patient has an array inside the larger array to index
import re

# Checks if input string is longer than 500 chars, if so then split into 3 GPT calls, one for each column,
# if not then do 1 GPT call and return result
def inputSplit(input_columns, que, index):
    while len(input_columns) < 3: # Some text columns may be empty so add a blank entry
        input_columns.append("")
        
    input_string = " ".join(map(str, input_columns)) # Apply str() to each index of input_columns then join with a space
    prompt = f"{input_string} \n {que}" # Create base prompt (input_columns + question)
    
    if(len(input_string) > 500):
        tmp_ans = [""] * len(input_columns)
        check_ans = [""] * len(input_columns)
        
        # Split into 3 GPT calls, one per column, and remove anything that isn't a number from the result
        for i in range(len(input_columns)):
            prompt = f"{input_columns[i]} \n {que}" # Recreate prompt with 1 column + question
            tmp_ans[i] = call_gpt(prompt) # Call GPT with prompt and save answer to array
            
            #print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
            
            check_ans[i] = re.split(r'\s+|-', tmp_ans[i]) # Split by spaces or hyphens
            check_ans[i] = [re.sub(r'[^0-9.]', '', j) for j in check_ans[i]] # Remove non numerical values
            check_ans[i] = [j for j in check_ans[i] if j.replace('.', '', 1).isdigit()] # Remove empty strings and isolated periods
            
            print(f"check_ans[{i}] = {str(check_ans[i])}")
            
        # Check if 2 arrays are the same output, if so return one of them if it is not empty
        if(check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]):
            if check_ans[0]:
                return tmp_ans[0]
            elif check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]
        elif(check_ans[1] == check_ans[2]):
            if check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]
        
        # If no matches, return first numerical string
        for i in range(len(check_ans)):
            if any(j not in ['', '.'] for j in check_ans[i]):
                return tmp_ans[i]
            
        # All non numerical answers, return empty string
        return ""
    else:
        # Call GPT once on all columns combined
        return call_gpt(prompt)
    

trueResults = [0 for i in range(sampleSize)]
trueResults_final = [0 for i in range(sampleSize)]

output = [0 for i in range(sampleSize)]
output_final = [0 for i in range(sampleSize)]

for i in range(0, min(sampleSize, len(Cat))):
    # If skip_patients is True and current index is in skip_patients_array, move to next index
    if skip_patients and i in skip_patients_array:
        continue
        
    trueResults[i] = Cat.at[i, 'Size at Greatest Dimension (S) ***Be  sure this is from the note and not the Mohs episode!']
    #que = "Question: what are the tumor sizes? answer in only an array of the numbers, followed by the units"
    que = "Question: What are the tumor sizes listed in the following patient note?"
    ans = ""
    
    if multiple_columns:
        input_columns = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
        ans = inputSplit(input_columns, que, i) # Result from GPT call
    elif not multiple_columns:
        input_columns = Cat.at[i, 'Physical Exam (PE)']
        prompt = f"{input_columns} \n {que}"
        if(i != 47):
            ans = call_gpt(prompt)
    
    print(f"Result[{i}] = {ans}")
    output[i] = ans

check_ans[0] = ['0.8', '0.8', '0.4', '0.5']
check_ans[1] = []
check_ans[2] = ['0.8', '0.8', '0.4', '0.5']
Result[0] = ["0.8 x 0.8cm", "0.4 x 0.5cm"]
check_ans[0] = ['0.6', '0.4']
check_ans[1] = ['0.6', '0.4']
check_ans[2] = ['0.7', '0.7']
Result[1] = ["0.6 x 0.4cm"]
check_ans[0] = ['3', '3']
check_ans[1] = []
check_ans[2] = []
Result[2] = ["3mm x 3mm"]
check_ans[0] = ['6.5', '5.0']
check_ans[1] = ['2']
check_ans[2] = []
Result[3] = ["6.5 x 5.0cm"]
check_ans[0] = ['0.6', '0.8', '1.2', '1.6', '1', '1', '2.5', '3.5']
check_ans[1] = ['1']
check_ans[2] = []
Result[4] = ["0.6 x 0.8 cm", "1.2 x 1.6 cm", "1 x 1 cm", "2.5 x 3.5 cm"]
check_ans[0] = ['0.4', '0.3']
check_ans[1] = []
check_ans[2] = []
Result[5] = ["0.4 x 0.3cm"]
check_ans[0] = ['0.3', '0.3']
check_ans[1] = []
check_ans[2] = []
Result[6] = ["0.3 x 0.3"]
check_ans[0] = ['1.5', '1', '8', '5']
check_ans[1] = []
check_ans[2] = ['0.9', '0.7', '0.2']
Result[9] = ["1.5cm x 1cm", "8mm x 5mm"]
check_ans[0] = ['1', '1.2']
check_ans[1] = []
ch

In [5]:
# Run previous code segment before running this one

# Reads through the array of patient tumor size arrays to find the largest dimension of tumor size for each patient
# Unit Filter: Checks for 'mm' and 'millimeters' and sets a flag to divide numbers by 10 later to convert from mm to cm

# Convert units while making sure num not smaller than decimal tenths place (0.03 -> 0.3)
def convert_size(val, unit):
    if unit in ['mm', 'millimeters']:  
        val /= 10  # Convert mm to cm
    
    if val < 0.1:  
        val *= 10  # Ensure at least one decimal place (tenths)
    
    return round(val, 2)  # Round to 2 decimal places


import re

mmArr = [0 for _ in range(len(output))]

for i in range(0, min(sampleSize, len(Cat))):
    if skip_patients and i in skip_patients_array:
        continue

    sizes = output[i]
    print(f"i={i} sizes: {sizes}")
    sizesReal = str(trueResults[i])
    
    size_matches = re.findall(
        r'(\d*\.?\d+)\s*to\s*(\d*\.?\d+)?\s*(cm|mm|millimeters)?'  # Matches "8 to 10 mm"
        r'|(\d*\.?\d+)\s*(cm|mm|millimeters)?\s*[x×]?\s*(\d*\.?\d+)?\s*(cm|mm|millimeters)?' # Matches "3 x 3 mm"
        r'|(\d*\.?\d+)\s*-\s*(\d*\.?\d+)\s*(cm|mm|millimeters)?',  # Matches "2-3 mm"
        sizes, 
        re.IGNORECASE
    )

    numeric_sizes = []
    
    for match in size_matches:
        print(f"i={i} match: {match}")
        # "8 to 10 mm" case
        if match[0]:  
            val1 = convert_size(float(match[0]), match[2])
            val2 = convert_size(float(match[1]), match[2]) if match[1] else None
            numeric_sizes.append(max(val1, val2) if val2 else val1)

        # "4 x 5 mm" case
        elif match[3]:  
            val1 = convert_size(float(match[3]), match[4])
            val2 = convert_size(float(match[5]), match[6]) if match[5] else val1
            numeric_sizes.append(max(val1, val2))

        # "2-3 mm" case
        elif match[7]:  
            val1 = convert_size(float(match[10]), match[12])
            val2 = convert_size(float(match[11]), match[12])
            numeric_sizes.append(max(val1, val2))
            print(f"i={i} numeric_sizes: {numeric_sizes}")  # Output the extracted sizes

    # Determine if we need to convert all to cm
    if 'mm' in sizes.lower() or 'millimeters' in sizes.lower():
        if all(num >= 1 for num in numeric_sizes):
            mmArr[i] = 1  # Mark for conversion

    # Convert if necessary
    if len(numeric_sizes) == 0:
        largest_size = -2  # No valid tumor size detected
    else:
        if mmArr[i] == 1:
            largest_size = max(numeric_sizes) / 10  # Convert mm to cm
        else:
            largest_size = max(numeric_sizes)  # Already in cm

        largest_size = round(largest_size, 2)  # Round to 2 decimal places

    # Process the real size values
    if sizesReal.strip():
        sizesReal_parts = re.split(r'[;/]', sizesReal)
        valid_sizes = []
        
        for part in sizesReal_parts:
            part = part.strip()
            if re.search(r'NA|nan', part, re.IGNORECASE):
                continue
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', part):
                continue
            num_match = re.search(r'(\d*\.?\d+)', part)
            if num_match:
                valid_sizes.append(float(num_match.group(1)))
        
        largest_size_real = max(valid_sizes) if valid_sizes else -1

    else:
        largest_size_real = -1  # No valid tumor size provided in trueResults

    # Store results
    output_final[i] = largest_size
    trueResults_final[i] = largest_size_real

    print(f"Result[{i}]: ChatGPT Prediction = {output_final[i]} Greatest Dimension Given = {trueResults_final[i]}")

# Need to split by cm and mm a different way due to cases like these
# 1 cm x 7 mm, 6 mm x 5 mm     Result[158] predicted = 0.7 real = 1.0
# 0.6 x 0.8 cm, 1.2 x 1.6 cm, 1 x 1 cm, 2.5 x 3.5 cm, cm     Result[4] predicted = 3.5 real = 3.5
# [6.5, 5.0] cm

i=0 sizes: ["0.8 x 0.8cm", "0.4 x 0.5cm"]
i=0 match: ('', '', '', '0.8', '', '0.8', 'cm', '', '', '')
i=0 match: ('', '', '', '0.4', '', '0.5', 'cm', '', '', '')
Result[0]: ChatGPT Prediction = 0.8 Greatest Dimension Given = 0.8
i=1 sizes: ["0.6 x 0.4cm"]
i=1 match: ('', '', '', '0.6', '', '0.4', 'cm', '', '', '')
Result[1]: ChatGPT Prediction = 0.6 Greatest Dimension Given = 0.6
i=2 sizes: ["3mm x 3mm"]
i=2 match: ('', '', '', '3', 'mm', '3', 'mm', '', '', '')
Result[2]: ChatGPT Prediction = 0.3 Greatest Dimension Given = 0.3
i=3 sizes: ["6.5 x 5.0cm"]
i=3 match: ('', '', '', '6.5', '', '5.0', 'cm', '', '', '')
Result[3]: ChatGPT Prediction = 6.5 Greatest Dimension Given = 6.5
i=4 sizes: ["0.6 x 0.8 cm", "1.2 x 1.6 cm", "1 x 1 cm", "2.5 x 3.5 cm"]
i=4 match: ('', '', '', '0.6', '', '0.8', 'cm', '', '', '')
i=4 match: ('', '', '', '1.2', '', '1.6', 'cm', '', '', '')
i=4 match: ('', '', '', '1', '', '1', 'cm', '', '', '')
i=4 match: ('', '', '', '2.5', '', '3.5', 'cm', '', '', '')
Resul

In [6]:
total_size = 0
correct_predictions = 0
no_size_given = 0
no_numerical_result_returned = 0
bad_cases_combined = 0
nsg_not_nnrr = 0
nnrr_not_nsg = 0
# nsg = no size given in Greastest Dimension column
# nnrr = no numerical result returned from ChatGPT output

for i in range(0, min((sampleSize), len(Cat))):
    if(float(output_final[i]) == 0 and float(trueResults_final[i]) == -1):
        bad_cases_combined += 1
        
    if(float(output_final[i]) == -2):
        no_numerical_result_returned += 1
        if(float(trueResults_final[i]) != -1):
            nnrr_not_nsg += 1
            
    if(float(trueResults_final[i]) == -1):
        no_size_given += 1
        if(float(output_final[i]) != 0):
            nsg_not_nnrr += 1
            
    if(float(output_final[i]) == float(trueResults_final[i])):
        correct_predictions += 1
        
    total_size += 1
    #print("Result[" + str(i) + "] predicted = " + str(output_final[i]))
    #print("Result[" + str(i) + "] true = " + str(trueResults_final[i]))
    #print()

# Accuracy when not removing bad cases
accuracy = correct_predictions / total_size
print("# of Correct Predictions = " + str(correct_predictions))
print("Sample Size = " + str(total_size))
print("Accuracy = " + "{:.2f}%".format(accuracy * 100))

# Accuracy when removing cases where the Greatest Dimension is not given (best to look at)
no_size_given_accuracy = correct_predictions / (total_size - no_size_given)
print()
print("Number of Notes without a Given Size = " + str(no_size_given))
print("Accuracy = " +"{:.2f}%".format(no_size_given_accuracy * 100))

# Accuracy when removing cases where GPT didn't return a numerical answer (not the best to look at, could be slightly lower)
no_numerical_result_returned_accuracy = correct_predictions / (total_size - no_numerical_result_returned)
print()
print("Number of PE/A&P/A Notes without a GPT Given Size = " + str(no_numerical_result_returned))
print("Accuracy = " + "{:.2f}%".format(no_numerical_result_returned_accuracy * 100))

# Accuracy when removing all above bad cases (ok to look at, could be slightly lower)
bad_cases_total_accuracy = correct_predictions / (total_size - (nnrr_not_nsg + nsg_not_nnrr + bad_cases_combined))
print()
print("Number of Combined Notes without a Given Size = " + str(nnrr_not_nsg + nsg_not_nnrr + bad_cases_combined))
print("Accuracy = " + "{:.2f}%".format(bad_cases_total_accuracy * 100))

# of Correct Predictions = 216
Sample Size = 294
Accuracy = 73.47%

Number of Notes without a Given Size = 74
Accuracy = 98.18%

Number of PE/A&P/A Notes without a GPT Given Size = 39
Accuracy = 84.71%

Number of Combined Notes without a Given Size = 74
Accuracy = 98.18%


In [195]:
for i in range(0, min((sampleSize), len(Cat))):
    if float(output_final[i]) != float(trueResults_final[i]) and float(trueResults_final[i]) != -1.0:
        print(f"i = {i}: Unfiltered ChatGPT Prediction = {output[i]}\nUnfiltered Greatest Dimension Given = {trueResults[i]}")
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} Greatest Dimension Given = {trueResults_final[i]}")
        print()
        print("-"*40) # Spacer
# Incorrect
# 134, 290         "4 x 5 mm" -> 0.4                        (couldn't figure this one out)
#                  "[4 x 5 mm]" -> 4.0                      (same issue as first one most likely)
# 237              "[8, 8, 0.9, 0.8] cm" -> 8.0             (add a filter whre if # > 7 then divide it)
# 97,              "[0.5, 1] cm" -> 1.0                     (1 is supposed to be in mm, GPT simply got it wrong, try preprocessing once regex is fixed)
# 116, 100         "2-3mm" -> 2.0                           (same issue as first one most likely)
# 254              "[0.2, 0.2, cm, 1, mm, 1, mm]" -> 1.0    (weird segmentation)
# 251              "[0.3, 0.3, cm], [1, mm]" -> 1.0         (this one is unique i don't wanna think about it rn)
# 231 should've been predicted correctly but the A and A&P matching caused it to be wrong, waiting on Andrew's response for clarification on those cols

i = 97: Unfiltered ChatGPT Prediction = [0.5, 1] cm
Unfiltered Greatest Dimension Given = 0.5
i = 97: ChatGPT Prediction = 1.0 Greatest Dimension Given = 0.5

----------------------------------------
i = 100: Unfiltered ChatGPT Prediction = [8-10 mm, 4-5 mm, 3-4 mm, 4-5 mm]
Unfiltered Greatest Dimension Given = 1.0/0.5/0.4
i = 100: ChatGPT Prediction = 8.0 Greatest Dimension Given = 1.0

----------------------------------------
i = 116: Unfiltered ChatGPT Prediction = 2-3 mm
Unfiltered Greatest Dimension Given = 0.3
i = 116: ChatGPT Prediction = 2.0 Greatest Dimension Given = 0.3

----------------------------------------
i = 134: Unfiltered ChatGPT Prediction = 4 x 5 mm
Unfiltered Greatest Dimension Given = 0.5
i = 134: ChatGPT Prediction = 0.4 Greatest Dimension Given = 0.5

----------------------------------------
i = 231: Unfiltered ChatGPT Prediction = [<1, 2] cm
Unfiltered Greatest Dimension Given = 0.5
i = 231: ChatGPT Prediction = 2.0 Greatest Dimension Given = 0.5

------------

In [196]:
index = 231
note = Cat.at[index, 'Physical Exam (PE)']
given_size = Cat.at[index, 'Size at Greatest Dimension (S) ***Be  sure this is from the note and not the Mohs episode!']
print(f"i = {index}: Given Greatest Dimension = {given_size}")
print(f"Unfiltered ChatGPT Prediction = {output[index]} ChatGPT Prediction = {output_final[index]}\n{note}")
# Incorrect Predictions:
# 51 (cm vs mm issue), 92, 114, 115, 125, 158, 183, 207, 237, 251, 254, 274 (???), 278 (6x4mm -> 6.4), 290
# Only issue is how mm vs cm is handled, need to fix the reg ex for that

i = 231: Given Greatest Dimension = 0.5
Unfiltered ChatGPT Prediction = [<1, 2] cm ChatGPT Prediction = 2.0
Physical exam:
General: Well-dressed; well-nourished.
Psych: Pleasant affect.
Neuro: Alert and oriented to person.
Skin: Total skin excluding undergarment areas, which includes the head/face, neck, left and right arms, chest, back, abdomen, left and right legs, digits and/or nails, was examined
- There are variegated hypo- and hyper-pigmented macules scattered over sun exposed areas on the chest, back, and neck. 
- There are tan to brown waxy stuck-on appearing papule(s) on the trunk and extremities.
- There are numerous cherry red dome-shaped papules overlying the anterior trunk, posterior trunk and bilateral upper and bilateral lower extremities.
- There are erythematous macule(s) with overlying adherent scale on the left posterior shoulder.
- There is a keratotic, erythematous macule on the left anterior chest.
- There is a 0.5 x 0.5 keratotic eroded papule on the left lateral